In [1]:
# Imports and setup
from pydrake.all import *
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.meshcat_utils import AddMeshcatTriad
from pathlib import Path
import numpy as np
import sys

sys.path.append(str(Path.cwd()))
from throw_helpers import interpolate_poses_linear, create_q_knots

In [2]:
# Start meshcat
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
# Scene configuration
TABLE_SDF_PATH = f"{Path.cwd()}/assets/table.sdf"

# Ball position: in front of robot, on table surface
BALL_POS = [0, -0.5, 0.025]  # x=side, y=forward, z=up

scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{TABLE_SDF_PATH}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: ball
    file: package://drake_models/manipulation_station/sphere.sdf
    default_free_body_pose:
        base_link:
            translation: {BALL_POS}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

In [4]:
# Build and visualize initial scene
meshcat.Delete()

builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# Get initial poses
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

X_WG_initial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
ball_body = plant.GetBodyByName("base_link", plant.GetModelInstanceByName("ball"))
X_WBall = plant.EvalBodyPoseInWorld(temp_plant_context, ball_body)

print(f"Gripper initial pose: {X_WG_initial}")
print(f"Ball pose: {X_WBall}")

# Add frame triad attached to gripper body (moves with gripper during simulation)
AddFrameTriadIllustration(scene_graph=scene_graph, body=plant.GetBodyByName("body"), length=0.08, radius=0.004)

# Build diagram and publish to meshcat (no simulation yet)
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

Gripper initial pose: RigidTransform(
  R=RotationMatrix([
    [0.999999682931835, 0.00019052063137842197, -0.0007731999219133521],
    [0.0007963267107334455, -0.2392492533556365, 0.9709578572896667],
    [1.8684985011115333e-16, -0.9709581651495918, -0.23924932921398262],
  ]),
  p=[0.0003707832187589982, -0.46561680802324634, 0.6793215789060889],
)
Ball pose: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, -0.5, 0.025],
)


In [5]:
# Pick trajectory parameters
GRIPPER_OPEN = 0.107
GRIPPER_CLOSED = 0.0
GRASP_HEIGHT = 0.12  # Height above ball center to grasp
APPROACH_HEIGHT = 0.1  # Height above grasp pose for approach

def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    """Grasp pose above object with gripper pointing down."""
    return X_WO @ RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, GRASP_HEIGHT])

def make_pick_trajectory(
    X_WG_start,
    X_WBall,
    t_approach=1.0,
    t_descend=0.5,
    t_grasp=0.5,
    t_lift=0.5,
):
    """
    Generate pick trajectory: approach -> descend -> grasp -> lift.
    Returns: (q_traj, gripper_traj, total_time, X_WG_end)
    """
    X_WG_grasp = design_grasp_pose(X_WBall)
    X_WG_approach = RigidTransform(p=[0, 0, APPROACH_HEIGHT]) @ X_WG_grasp
    
    # Keyframe times
    t0, t1, t2, t3 = 0, t_approach, t_approach + t_descend, t_approach + t_descend + t_grasp
    t4 = t3 + t_lift
    
    # Pose trajectory: start -> approach -> grasp -> hold -> lift back to approach
    pose_traj = PiecewisePose.MakeLinear(
        [t0, t1, t2, t3, t4],
        [X_WG_start, X_WG_approach, X_WG_grasp, X_WG_grasp, X_WG_approach]
    )
    
    # Gripper trajectory: open -> open -> close -> close -> close
    gripper_traj = PiecewisePolynomial.FirstOrderHold(
        [t0, t1, t2, t3, t4],
        np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED]])
    )
    
    # Convert to joint trajectory via IK
    num_samples = 50
    t_samples = np.linspace(t0, t4, num_samples)
    poses = [pose_traj.GetPose(t) for t in t_samples]
    q_knots = create_q_knots(plant, poses)
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_samples[:len(q_knots)], np.array(q_knots).T)
    
    return q_traj, gripper_traj, t4, X_WG_approach

In [6]:
# Generate pick trajectory
q_traj, gripper_traj, total_time, X_WG_postpick = make_pick_trajectory(X_WG_initial, X_WBall)
print(f"Pick trajectory: {total_time:.1f}s, {q_traj.get_number_of_segments()} segments")

Pick trajectory: 2.5s, 49 segments


In [8]:
# Throw pose design
def design_throw_poses(theta, prethrow_radius=0.35, prethrow_height=0.4, arc_radius=0.15):
    """
    Design prethrow and throw poses for a given heading angle.
    Args:
        theta: Direction to throw (radians, 0 = +x, pi/2 = +y)
        prethrow_radius: Distance from origin in xy-plane for wind-up
        prethrow_height: Height of wind-up pose
        arc_radius: Radius of throwing arc (determines throw pose offset)
    Returns: (T_world_prethrow, T_world_throw)
    """
    # Prethrow: gripper points along throw direction
    T_world_prethrow = RigidTransform(
        RotationMatrix.MakeXRotation(-np.pi/2) @ RotationMatrix.MakeYRotation(theta - np.pi/2),
        [prethrow_radius * np.cos(theta), prethrow_radius * np.sin(theta), prethrow_height],
    )
    
    # Throw: forward and up along arc
    T_world_throw = RigidTransform(
        RotationMatrix.MakeXRotation(-np.pi/2) @ RotationMatrix.MakeYRotation(theta - np.pi/2) @ RotationMatrix.MakeXRotation(-np.pi/2),
        T_world_prethrow.translation() + [arc_radius * np.cos(theta), arc_radius * np.sin(theta), arc_radius],
    )
    
    return T_world_prethrow, T_world_throw

In [12]:
# Visualize throw poses
P_TARGET = np.array([0, -1.0, 0.0])  # Target on ground plane
theta_throw = np.arctan2(P_TARGET[1], P_TARGET[0])

T_world_prethrow, T_world_throw = design_throw_poses(theta_throw)

AddMeshcatTriad(meshcat, "prethrow", length=0.15, radius=0.005, X_PT=T_world_prethrow)
AddMeshcatTriad(meshcat, "throw", length=0.15, radius=0.005, X_PT=T_world_throw)
AddMeshcatTriad(meshcat, "target", length=0.2, radius=0.008, X_PT=RigidTransform(p=P_TARGET))

print(f"Throw direction: {np.degrees(theta_throw):.1f}°")
print(f"Prethrow: {T_world_prethrow.translation()}")
print(f"Throw: {T_world_throw.translation()}")

Throw direction: -90.0°
Prethrow: [ 2.1431319e-17 -3.5000000e-01  4.0000000e-01]
Throw: [ 3.061617e-17 -5.000000e-01  5.500000e-01]


In [13]:
# Throw trajectory
from throw_helpers import interpolate_poses_arc_motion

def make_throw_trajectory(
    X_WG_start,
    p_target,
    t_windup=1.0,
    t_throw=0.3,
    release_frac=0.7,
):
    """
    Generate throw trajectory: windup -> arc throw with release.
    Args:
        X_WG_start: Starting gripper pose
        p_target: Target position [x, y, z]
        t_windup: Time to reach prethrow pose
        t_throw: Time for throwing arc
        release_frac: Fraction along arc to release (0-1)
    Returns: (q_traj, gripper_traj, total_time)
    """
    theta = np.arctan2(p_target[1], p_target[0])
    T_prethrow, T_throw = design_throw_poses(theta)
    
    # Keyframe times
    t0, t1, t2 = 0, t_windup, t_windup + t_throw
    t_release = t_windup + t_throw * release_frac
    
    # Generate dense pose samples for IK
    num_windup = 30
    num_throw = 30
    
    # Windup: linear interpolation to prethrow
    t_windup_samples = np.linspace(t0, t1, num_windup, endpoint=False)
    poses_windup = [
        interpolate_poses_linear(X_WG_start, T_prethrow, t / t_windup)
        for t in t_windup_samples
    ]
    
    # Throw: arc motion from prethrow to throw
    t_throw_samples = np.linspace(t1, t2, num_throw)
    poses_throw = [
        interpolate_poses_arc_motion(T_prethrow, T_throw, (t - t1) / t_throw)
        for t in t_throw_samples
    ]
    
    all_poses = poses_windup + poses_throw
    all_times = np.concatenate([t_windup_samples, t_throw_samples])
    
    # Convert to joint trajectory
    q_knots = create_q_knots(plant, all_poses)
    q_traj = PiecewisePolynomial.CubicShapePreserving(all_times[:len(q_knots)], np.array(q_knots).T)
    
    # Gripper: closed until release, then open
    gripper_traj = PiecewisePolynomial.FirstOrderHold(
        [t0, t_release, t_release + 0.001, t2],
        np.array([[GRIPPER_CLOSED, GRIPPER_CLOSED, GRIPPER_OPEN, GRIPPER_OPEN]]),
    )
    
    return q_traj, gripper_traj, t2

In [14]:
# Generate throw trajectory (starting from post-pick pose)
q_traj_throw, gripper_traj_throw, throw_time = make_throw_trajectory(X_WG_postpick, P_TARGET)
print(f"Throw trajectory: {throw_time:.2f}s, {q_traj_throw.get_number_of_segments()} segments")

IK failed at pose 46/60: IK failed to find solution for pose: RigidTransform(
  R=RotationMatrix([
    [-1.0, 9.33379321243629e-17, -7.928195414699615e-17],
    [1.2246467991473532e-16, 0.7621620551276369, -0.6473862847818271],
    [6.162975822039155e-33, -0.6473862847818271, -0.7621620551276369],
  ]),
  p=[2.84316638944059e-17, -0.4643243082691455, 0.45289205728272597],
)
Throw trajectory: 1.30s, 45 segments
